In [2]:
import copy
import numpy as np

In [3]:
np.random.seed(0)

In [4]:
# defining the sigmoid function and itsderivative
def sigmoid(x,der = False):
    if(der == False):
        return 1/(1+np.exp(-x))
    else:
        return x*(1-x)

In [5]:
#initialising the integer to binary table
int2binary = {}
binary_dim = 8

largestNo = pow(2,binary_dim)
binary = np.unpackbits(np.array([range(largestNo)],dtype=np.uint8).T,axis=1)
for i in range(largestNo):
    int2binary[i] = binary[i]

In [6]:
#input variables
input_dim = 2
output_dim = 1
hidden_dim = 16
alpha = 0.1

In [7]:
#initialising the synapses(weights)
synapse0 = 2*np.random.rand(input_dim,hidden_dim) - 1
synapse1 = 2*np.random.rand(hidden_dim,output_dim) - 1
synapseH = 2*np.random.rand(hidden_dim,hidden_dim) - 1

In [8]:
#updating the synapses
synapse0_update = np.zeros_like(synapse0)
synapse1_update = np.zeros_like(synapse1)
synapseH_update = np.zeros_like(synapseH)

In [9]:
for iter in range(10):
    a_int = np.random.randint(largestNo/2)
    a = int2binary[a_int]
    
    b_int = np.random.randint(largestNo/2)
    b = int2binary[b_int]
    
    c_int = a_int + b_int
    c = int2binary[c_int]
    
    d = np.zeros_like(c)
    
    overallError = 0
    
    layer2deltas = list()
    layer1values = list()
    layer1values.append(np.zeros(hidden_dim))
    
    for i in range(binary_dim):
        #input array
        x = np.array([[a[binary_dim-1-i],b[binary_dim-1-i]]])
        
        #output array
        y = np.array([[c[binary_dim-1-i]]]).T
        
        #hidden layer
        layer1 = sigmoid(np.dot(x,synapse0) + np.dot(layer1values[-1],synapseH),False)
        
        #output layer
        layer2 = sigmoid(np.dot(layer1,synapse1),False)
        
        layer2error = y - layer2
        layer2deltas.append(layer2error*sigmoid(layer2,True))
        overallError += np.abs(layer2error[0])
        
        d[binary_dim-1-i] = np.round(layer2[0][0])
        
        layer1values.append(copy.deepcopy(layer1))
        #exit for loop

    futureLayer1delta = np.zeros(hidden_dim)

    # backtracking in neural network
    for i in range(binary_dim):
        x = np.array([[a[i],b[i]]])
        layer1 = layer1values[-i-1]
        prevLayer1 = layer1values[-i-2]
        
        layer2delta = layer2deltas[-i-1]
        layer1delta = (futureLayer1delta.dot(synapseH.T) + layer2delta.dot(synapse1.T)) * sigmoid(layer1,True)
        
        #update the synapses
        synapse1_update += np.atleast_2d(layer1).T.dot(layer2delta)
        synapseH_update += np.atleast_2d(prevLayer1).T.dot(layer1delta)
        synapse0_update += x.T.dot(layer1delta)
        
        futureLayer1delta = layer1delta
    
    synapse0 += synapse0_update*alpha
    synapse1 += synapse1_update*alpha
    synapseH += synapseH_update*alpha
    
    synapse0_update *= 0
    synapse1_update *= 0
    synapseH_update *= 0
    
    if(iter%200 == 0):
        print "Error:" + str(overallError)
        print "Prediction:" + str(d)
        print "Actual:" + str(c)
        out = 0
        for index,x in enumerate(reversed(d)):
            out += x*pow(2,index)
        print str(a_int) + " + " + str(b)